## Commonsense MCQA

This notebook provides a walkthrough of building a benchmark for steering improved performance on the [CommonsenseQA](https://huggingface.co/datasets/tau/commonsense_qa) problem set. The benchmark will compare three steering pipelines: the unsteered behavior (baseline model), few shot steering, and steering via a LoRA adapter.

For convenience, change the current directory to the notebook if necessary:

In [1]:
import os
os.chdir("./examples/notebooks/benchmark_commonsense_mcqa/")

## Building the use case

The use case of interest has already been constructed via the [use case](../../../docs/tutorials/add_new_use_case.md) tutorial and is available at `aisteer360/evaluation/use_cases/commonsense_mcqa/use_case.py`. It is initialized as follows:

In [ ]:
from aisteer360.evaluation.use_cases.commonsense_mcqa.use_case import CommonsenseMCQA
from aisteer360.evaluation.metrics.custom.commonsense_mcqa.mcqa_accuracy import MCQAAccuracy
from aisteer360.evaluation.metrics.custom.commonsense_mcqa.mcqa_positional_bias import MCQAPositionalBias

commonsense_mcqa = CommonsenseMCQA(
    evaluation_data="./data/evaluation_qa.jsonl",
    evaluation_metrics=[
        MCQAAccuracy(),
        MCQAPositionalBias(),
    ],
    num_shuffling_runs=20,
    num_samples=50  # optional
)

/dccstor/principled_ai/users/erikmiehling/AISteer360/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Two custom metrics have been created for the use case: `MCQAAccuracy` which measures the accuracy statistics of each question (across trials), and `MCQAPositionalBias` which measures the positional bias (via deviation from the uniform distribution across runs). To facilitate computation of these statistics, the use case accepts a keyword argument `num_shuffling_runs` dictating how many times each question should be presented to the (steered) model under a randomized ordering of the choices. The `num_samples` parameter dictates how many entries from `evaluation_data` are used during benchmarking.

## Defining the controls

The benchmark aims to compare two controls using common steering data.

In [3]:
import json

steering_data_path = "data/steer_qa.jsonl"

with open(steering_data_path, "r") as f:
    steering_data = [json.loads(line) for line in f]

steering_data[0]

{'id': '01beaf20-82aa-40b0-8b08-ee08b94e6666',
 'question': 'The spirit ascended to the after life, so what was it leaving?',
 'answer_chosen': 'human being',
 'answer_rejected': 'cemetary'}

The steering data consists of triples `(question, answer_chosen, answer_rejected)` extracted from the CommonsenseQA dataset where `answer_chosen` is the ground-truth answer and `answer_rejected` is a randomly selected incorrect answer. Both controls (`FewShot` and `DPO`) are based on the same steering data.

### Defining the few shot control

The `FewShot` control requires specification of example pools. As shown below, each positive example is given by the pair (`question`,`answer_chosen`) whereas each negative example is given by the pair (`question`,`answer_rejected`).

In [4]:
positive_pool = []
negative_pool = []
for row in steering_data:
    positive_pool.append({
        "question": row["question"],
        "answer": row["answer_chosen"]
    })
    negative_pool.append({
        "question": row["question"],
        "answer": row["answer_rejected"]
    })

These pools are then passed in to the `FewShot` class upon instantiation, along with the name of the example selector (how examples are drawn from the pools; defaults to `random`), and the counts for how many positive and negative examples the selector should draw from the pool.

In [5]:
from aisteer360.algorithms.input_control.few_shot.control import FewShot

few_shot = FewShot(
    selector_name="random",
    positive_example_pool=positive_pool,
    negative_example_pool=negative_pool,
    k_positive=25,
    k_negative=25
)

### Defining the DPO (with LoRA) control



In [6]:
from datasets import Dataset
from peft import PeftType
from aisteer360.algorithms.structural_control.wrappers.trl.dpotrainer.control import DPO


train_examples = []
for row in steering_data:
    train_examples.append({
        "prompt": row['question'],
        "chosen": row['answer_chosen'],  
        "rejected": row['answer_rejected']
    })
train_ds = Dataset.from_list(train_examples)

# instantiate dpo control
dpo_lora = DPO(
    train_dataset=train_ds,

    # DPO / TRL config
    output_dir="trl_models/Qwen2.5-0.5B-DPO-Lora-Steer",
    per_device_train_batch_size=4,
    num_train_epochs=2,
    learning_rate=1e-6,
    beta=0.1,
    loss_type="sigmoid", 
    max_length=1024,
    max_prompt_length=512,
    disable_dropout=True,
    logging_steps=100,
    save_strategy="no",
    report_to="none",
    seed=123,

    # LoRA config
    use_peft=True,
    peft_type=PeftType.LORA,
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    adapter_name="dpo",
    merge_lora_after_train=False,
)

## Instantiating (and running) the benchmark

Given the controls, the benchmark can now be run on any control pipelines, i.e., sequence of controls. In the following benchmark, we compare the unsteered baseline behavior (no control) with few-shot and DPO (with LoRA).

In [7]:
import transformers
from aisteer360.evaluation.benchmark import Benchmark
transformers.logging.set_verbosity_error()

benchmark = Benchmark(
    use_case=commonsense_mcqa,
    base_model_name_or_path="Qwen/Qwen2.5-1.5B-Instruct",
    steering_pipelines={
        "baseline": [],  # no steering
        "few_shot": [few_shot],
        "dpo_lora": [dpo_lora],
    },
    gen_kwargs={
        "max_new_tokens": 300,
        "do_sample": True,
        "temperature": 0.7,
    },
    device_map="auto"
)

# run and plot/export
profiles = benchmark.run()

Running pipeline: baseline...


done.
Running pipeline: few_shot...
done.
Running pipeline: dpo_lora...


Train dataset reference log probs: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1218/1218 [06:10<00:00,  3.29it/s]


{'loss': 0.693, 'grad_norm': 0.6648241281509399, 'learning_rate': 9.5935960591133e-07, 'rewards/chosen': 0.0010058283805847168, 'rewards/rejected': 0.0007286262698471546, 'rewards/accuracies': 0.6625000238418579, 'rewards/margins': 0.00027720213984139264, 'logps/chosen': -42.759090423583984, 'logps/rejected': -44.94538879394531, 'logits/chosen': 0.7523290514945984, 'logits/rejected': 0.834088921546936, 'epoch': 0.08210180623973727}
{'loss': 0.6928, 'grad_norm': 0.4628775417804718, 'learning_rate': 9.183087027914613e-07, 'rewards/chosen': 0.002631545066833496, 'rewards/rejected': 0.0019237594678997993, 'rewards/accuracies': 0.6600000262260437, 'rewards/margins': 0.0007077856571413577, 'logps/chosen': -42.33175277709961, 'logps/rejected': -45.524879455566406, 'logits/chosen': 0.7863448858261108, 'logits/rejected': 0.8765722513198853, 'epoch': 0.16420361247947454}
{'loss': 0.6925, 'grad_norm': 0.5489557385444641, 'learning_rate': 8.772577996715927e-07, 'rewards/chosen': 0.0041033243760466

In [8]:
benchmark.export(profiles, save_dir="./profiles/")

## Inspecting the profiles

Each control pipeline in the benchmark yields an evaluation profile. Each evaluation profile contains metric values as computed by the metrics passed in to the use case, in this case `MCQAAccuracy` and `MCQAPositionalBias`.

In [9]:
import json
print(json.dumps(profiles['baseline']['evaluations'], indent=2))

{
  "MCQAAccuracy": {
    "trial_mean": 0.61,
    "trial_std": 0.49020713000019756,
    "question_mean": 0.6,
    "question_std": 0.5477225575051662
  },
  "MCQAPositionalBias": {
    "mean": 0.12000000000000002,
    "std": 0.1013903348450926
  }
}


In [10]:
print(json.dumps(profiles['few_shot']['evaluations'], indent=2))

{
  "MCQAAccuracy": {
    "trial_mean": 0.93,
    "trial_std": 0.256432399976243,
    "question_mean": 1.0,
    "question_std": 0.0
  },
  "MCQAPositionalBias": {
    "mean": 0.023999999999999994,
    "std": 0.01788854381999832
  }
}


In [11]:
print(json.dumps(profiles['dpo_lora']['evaluations'], indent=2))

{
  "MCQAAccuracy": {
    "trial_mean": 0.65,
    "trial_std": 0.47937248544110195,
    "question_mean": 0.8,
    "question_std": 0.44721359549995804
  },
  "MCQAPositionalBias": {
    "mean": 0.10800000000000001,
    "std": 0.1063954886261631
  }
}


We can see that `FewShot` (using 25 positive/negative examples) yields the best improvement over baseline. The `DPO` (with LoRA) control yields a marginal improvement over the baseline, likely because of the small (5k) steering dataset.